# Transfer Learning using ResNet50

## Imports and Preprocess

Import libraries

In [1]:
import numpy as np
import pandas as pd

import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow import set_random_seed
from keras.applications.resnet50 import ResNet50, preprocess_input as resnet50_preprocess
#from keras.applications.densenet import DenseNet201
#from keras.applications.mobilenet_v2 import MobileNetV2
#from keras.applications.nasnet import NASNetMobile
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint

from src.sampler import train_test_bootstrapper
from src.model_api import getPretrainedModel

Using TensorFlow backend.


Import data

In [2]:
with open('data/derived/data_train.pickle', 'rb') as file:
    data_train = pickle.load(file)
with open('data/derived/labels_train.pickle', 'rb') as file:
    labels_train = pickle.load(file)
print('Shape of data_train:', data_train.shape)
print('Length of labels_train:', len(labels_train))

Shape of data_train: (25361, 224, 224, 3)
Length of labels_train: 25361


Convert labels to integers for model training

In [3]:
label_encoder = LabelEncoder()
labels_train_encoded = label_encoder.fit_transform(labels_train)
print('Original labels: {}'.format(labels_train[:5]))
print('Encoded labels: {}'.format(labels_train_encoded[:5]))
print('Label encoder classes: {}'.format(label_encoder.classes_[:5]))

Original labels: ['w_025911c', 'new_whale', 'new_whale', 'new_whale', 'new_whale']
Encoded labels: [44  0  0  0  0]
Label encoder classes: ['new_whale' 'w_0003639' 'w_0003c59' 'w_0027efa' 'w_00289b1']


Compute sample size and number of classes

In [4]:
sample_size = len(labels_train_encoded)
num_classes = len(set(labels_train_encoded))
print('Sample size:', sample_size)
print('Number of clases:', num_classes)

Sample size: 25361
Number of clases: 5005


## Train ResNet50

ResNet Model 1 (Random Seed = 1, assuming no class imbalance)

In [5]:
filepath = 'models/weights-{epoch:02d}-{val_loss:.3f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', save_best_only=True, save_weights_only=True,
                             mode='min')
early_stop = EarlyStopping(patience=2, monitor='val_loss')

model = getPretrainedModel(ResNet50(include_top=False, weights='imagenet', pooling='max'),
                           num_classes=len(set(labels_train_encoded)),
                           percent_layers_freeze=0.1)

batch_size = 32
image_gen = ImageDataGenerator(rotation_range=20,
                               width_shift_range=.1,
                               height_shift_range=.1,
                               shear_range=0.5,
                               zoom_range=(0.9, 1.1),
                               fill_mode='constant',
                               horizontal_flip=True)

In [6]:
X_train, X_test, y_train, y_test = train_test_bootstrapper(data_train, labels_train_encoded,
                                                           bootstrapper_size=sample_size,
                                                           class_imbalance=False, random_state=1)
    
X_train = resnet50_preprocess(X_train)
X_test = resnet50_preprocess(X_test)
y_train = to_categorical(y_train, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)
    
image_generator_samples = image_gen.flow(X_train, y_train, batch_size=batch_size, seed=1)
    
np.random.seed(1)
set_random_seed(1)
train_size, epochs = X_train.shape[0], 10
hist = model.fit_generator(image_generator_samples, steps_per_epoch=2*train_size//epochs,
                           validation_data=(X_test, y_test), epochs=epochs, shuffle=True,
                           callbacks=[checkpoint, early_stop])

model_resnet50.save_weights('models/weights_resnet50_1.hdf5')

Epoch 1/10
5072/5072 [==============================] - 1229s 242ms/step - loss: 6.0779 - acc: 0.3824 - val_loss: 6.5845 - val_acc: 0.3789
Epoch 2/10
5072/5072 [==============================] - 1227s 242ms/step - loss: 5.4525 - acc: 0.3826 - val_loss: 6.5948 - val_acc: 0.3789
Epoch 3/10
5072/5072 [==============================] - 1220s 241ms/step - loss: 5.2665 - acc: 0.3834 - val_loss: 6.4763 - val_acc: 0.3802
Epoch 4/10
5072/5072 [==============================] - 1225s 242ms/step - loss: 5.2000 - acc: 0.3843 - val_loss: 6.5521 - val_acc: 0.3792
Epoch 5/10
5072/5072 [==============================] - 1219s 240ms/step - loss: 5.1670 - acc: 0.3847 - val_loss: 6.5551 - val_acc: 0.3802


ResNet Model 2 (Random Seed = 2, assuming no class imbalance)

In [7]:
filepath = 'models/weights-{epoch:02d}-{val_loss:.3f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', save_best_only=True, save_weights_only=True,
                             mode='min')
early_stop = EarlyStopping(patience=2, monitor='val_loss')

model = getPretrainedModel(ResNet50(include_top=False, weights='imagenet', pooling='max'),
                           num_classes=len(set(labels_train_encoded)),
                           percent_layers_freeze=0.1)

batch_size = 32
image_gen = ImageDataGenerator(rotation_range=20,
                               width_shift_range=.1,
                               height_shift_range=.1,
                               shear_range=0.5,
                               zoom_range=(0.9, 1.1),
                               fill_mode='constant',
                               horizontal_flip=True)

In [8]:
X_train, X_test, y_train, y_test = train_test_bootstrapper(data_train, labels_train_encoded,
                                                           bootstrapper_size=sample_size,
                                                           class_imbalance=False, random_state=2)
    
X_train = resnet50_preprocess(X_train)
X_test = resnet50_preprocess(X_test)
y_train = to_categorical(y_train, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)
    
image_generator_samples = image_gen.flow(X_train, y_train, batch_size=batch_size, seed=2)
    
np.random.seed(2)
set_random_seed(2)
train_size, epochs = X_train.shape[0], 10
hist = model.fit_generator(image_generator_samples, steps_per_epoch=2*train_size//epochs,
                           validation_data=(X_test, y_test), epochs=epochs, shuffle=True,
                           callbacks=[checkpoint, early_stop])

model_resnet50.save_weights('models/weights_resnet50_2.hdf5')

Epoch 1/10
5072/5072 [==============================] - 1224s 241ms/step - loss: 6.0719 - acc: 0.3831 - val_loss: 6.4888 - val_acc: 0.3830
Epoch 2/10
5072/5072 [==============================] - 1211s 239ms/step - loss: 5.3947 - acc: 0.3837 - val_loss: 6.4665 - val_acc: 0.3830
Epoch 3/10
5072/5072 [==============================] - 1220s 241ms/step - loss: 5.2035 - acc: 0.3831 - val_loss: 6.4416 - val_acc: 0.3838
Epoch 4/10
5072/5072 [==============================] - 1223s 241ms/step - loss: 5.1343 - acc: 0.3843 - val_loss: 6.4123 - val_acc: 0.3831
Epoch 5/10
5072/5072 [==============================] - 1215s 240ms/step - loss: 5.1077 - acc: 0.3841 - val_loss: 6.3856 - val_acc: 0.3836
Epoch 6/10
5072/5072 [==============================] - 1214s 239ms/step - loss: 5.1010 - acc: 0.3845 - val_loss: 6.4501 - val_acc: 0.3839
Epoch 7/10
5072/5072 [==============================] - 1214s 239ms/step - loss: 5.0899 - acc: 0.3849 - val_loss: 6.7038 - val_acc: 0.3760


ResNet Model 3 (Random Seed = 1, considering class imbalance)

In [9]:
filepath = 'models/weights-{epoch:02d}-{val_loss:.3f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', save_best_only=True, save_weights_only=True,
                             mode='min')
early_stop = EarlyStopping(patience=2, monitor='val_loss')

model = getPretrainedModel(ResNet50(include_top=False, weights='imagenet', pooling='max'),
                           num_classes=len(set(labels_train_encoded)),
                           percent_layers_freeze=0.1)

batch_size = 32
image_gen = ImageDataGenerator(rotation_range=20,
                               width_shift_range=.1,
                               height_shift_range=.1,
                               shear_range=0.5,
                               zoom_range=(0.9, 1.1),
                               fill_mode='constant',
                               horizontal_flip=True)

In [10]:
X_train, X_test, y_train, y_test = train_test_bootstrapper(data_train, labels_train_encoded,
                                                           bootstrapper_size=sample_size,
                                                           class_imbalance=True, random_state=1)
    
X_train = resnet50_preprocess(X_train)
X_test = resnet50_preprocess(X_test)
y_train = to_categorical(y_train, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)
    
image_generator_samples = image_gen.flow(X_train, y_train, batch_size=batch_size, seed=1)
    
np.random.seed(1)
set_random_seed(1)
train_size, epochs = X_train.shape[0], 10
hist = model.fit_generator(image_generator_samples, steps_per_epoch=2*train_size//epochs,
                           validation_data=(X_test, y_test), epochs=epochs, shuffle=True,
                           callbacks=[checkpoint, early_stop])

model_resnet50.save_weights('models/weights_resnet50_3.hdf5')

Epoch 1/10
5072/5072 [==============================] - 1236s 244ms/step - loss: 7.3512 - acc: 0.1885 - val_loss: 5.6464 - val_acc: 0.5752
Epoch 2/10
5072/5072 [==============================] - 1223s 241ms/step - loss: 6.7859 - acc: 0.1891 - val_loss: 4.7750 - val_acc: 0.5752
Epoch 3/10
5072/5072 [==============================] - 1216s 240ms/step - loss: 6.6516 - acc: 0.1894 - val_loss: 4.7412 - val_acc: 0.5750
Epoch 4/10
5072/5072 [==============================] - 1214s 239ms/step - loss: 6.5902 - acc: 0.1908 - val_loss: 4.6863 - val_acc: 0.5761
Epoch 5/10
5072/5072 [==============================] - 1216s 240ms/step - loss: 6.5636 - acc: 0.1903 - val_loss: 4.8199 - val_acc: 0.5752
Epoch 6/10
5072/5072 [==============================] - 1219s 240ms/step - loss: 6.5562 - acc: 0.1903 - val_loss: 4.7702 - val_acc: 0.5741


ResNet Model 4 (Random Seed = 2, considering class imbalance)

In [11]:
filepath = 'models/weights-{epoch:02d}-{val_loss:.3f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', save_best_only=True, save_weights_only=True,
                             mode='min')
early_stop = EarlyStopping(patience=2, monitor='val_loss')

model = getPretrainedModel(ResNet50(include_top=False, weights='imagenet', pooling='max'),
                           num_classes=len(set(labels_train_encoded)),
                           percent_layers_freeze=0.1)

batch_size = 32
image_gen = ImageDataGenerator(rotation_range=20,
                               width_shift_range=.1,
                               height_shift_range=.1,
                               shear_range=0.5,
                               zoom_range=(0.9, 1.1),
                               fill_mode='constant',
                               horizontal_flip=True)

In [12]:
X_train, X_test, y_train, y_test = train_test_bootstrapper(data_train, labels_train_encoded,
                                                           bootstrapper_size=sample_size,
                                                           class_imbalance=True, random_state=2)
    
X_train = resnet50_preprocess(X_train)
X_test = resnet50_preprocess(X_test)
y_train = to_categorical(y_train, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)
    
image_generator_samples = image_gen.flow(X_train, y_train, batch_size=batch_size, seed=2)
    
np.random.seed(2)
set_random_seed(2)
train_size, epochs = X_train.shape[0], 10
hist = model.fit_generator(image_generator_samples, steps_per_epoch=2*train_size//epochs,
                           validation_data=(X_test, y_test), epochs=epochs, shuffle=True,
                           callbacks=[checkpoint, early_stop])

model_resnet50.save_weights('models/weights_resnet50_4.hdf5')

Epoch 1/10
5072/5072 [==============================] - 1273s 251ms/step - loss: 7.3792 - acc: 0.1882 - val_loss: 5.5530 - val_acc: 0.5702
Epoch 2/10
5072/5072 [==============================] - 1234s 243ms/step - loss: 6.8389 - acc: 0.1891 - val_loss: 4.7747 - val_acc: 0.5703
Epoch 3/10
5072/5072 [==============================] - 1232s 243ms/step - loss: 6.6705 - acc: 0.1893 - val_loss: 5.0889 - val_acc: 0.5600
Epoch 4/10
5072/5072 [==============================] - 1230s 243ms/step - loss: 6.6142 - acc: 0.1894 - val_loss: 4.8610 - val_acc: 0.5652
